# Inversion : The Traditional Way

$$\mathbf{A^{-1}} = \dfrac{1}{|\mathbf{A}|} \text{adj(A})$$

In [1]:
# Imports and Auxiliary Functions
import numpy as np

def cofactor(A, m, n): # Function to find and return the cofactor matrix of a matrix
    shape = np.shape(A)
    newshape = (shape[0]-1, shape[1]-1) # Since we will exclude one row and one column
    B = []
    for i in range(shape[0]):
        for j in range(shape[1]):
            if(i != m and j != n):
                B.append(A[i][j]) # Linearly accumulate new elements
    return np.reshape(B, newshape) # Reshape to form new matrix

def determinant(A): # Function to calculate the determinants
    shape = np.shape(A)
    result = 0.
    if(shape[0] == 2):
        result = A[0][0] * A[1][1] - A[1][0] * A[0][1]
    else:
        for j in range(shape[1]):
            term = 1.
            term *= ((-1.)**j)*A[0][j]
            cofactor_matrix = cofactor(A, 0, j)
            term *= determinant(cofactor_matrix) # Recursive calculation
            result += term
    return result

In [2]:
# Traditional Inverse - Main Function

def traditional_matrix_inverse(A):
    delta = determinant(A)
    shape = np.shape(A)
    adj = np.zeros_like(A)
    for i in range(shape[0]):
        for j in range(shape[1]):
            adj[i][j] = (-1.)**(i + j) * determinant(cofactor(A, i, j))
    adj = adj.T
    inv = np.power(delta, -1) * adj
    return inv

---
# Inversion : The Non-traditional Way
We will solve the system of linear equations given by :
$$\mathbf{A}\vec{x}_j = \mathbf{I}_j \quad (\text{j }=1, \cdots, n)$$
where, $\mathbf{I}_j$ is the j$^{th}$ column of the n x n identity matrix, and the solutions to the above system defines the j$^{th}$ column of the inverse matrix.

In [3]:
# Non-traditional Inverse - Gaussian Elimination auxiliary function

def gaussian_eliminate(A, b):
    rows = np.shape(A)[0]
    columns = np.shape(A)[1]
    B = np.copy(A)
    c = np.copy(b)
    
    if(B[0][0] == 0):
        temp = B[0][:]
        B[0][:] = B[1][:]
        B[1][:] = temp
        temp = c[0]
        c[0] = c[1]
        c[1] = temp
        
    for i in range(rows):
        for j in range(i+1, rows):
            multfactor = B[j][i] / B[i][i]
            B[j][:] = B[j][:] - multfactor * B[i][:]
            c[j] = c[j] - multfactor * c[i]
            
    outputs = np.zeros(columns)
    
    for i in range(rows-1, -1, -1):
        summation = 0
        for j in range(columns-1, i, -1):
            summation += B[i][j] * outputs[j]
        answer = (c[i] - summation) / B[i][i]
        outputs[i] = answer
        
    return(outputs)

In [4]:
# Non-traditional Inverse - Main Function

def nontraditional_matrix_inverse(A):
    shape = np.shape(A)
    I = np.eye(shape[0])
    inv = np.zeros_like(A, dtype=float)
    for j in range(shape[1]):
        a = A
        b = I[:][j]
        inv[:][j] = gaussian_eliminate(a, b) #currently using Gaussian Elimination code
    return inv.T

---
# Inversion : Using the Faddeev-LeVerrier Algorithm
In the Faddeev-LeVerrier Algorithm, we define the auxiliary matrices and the coefficients as follows:
$$\mathbf{M}_0 = \mathbf{0}; \quad c_n = 1 \quad( k = 0 )$$

$$\mathbf{M}_k = \mathbf{AM}_{k-1} + c_{n-k+1}\cdot \mathbf{I}; \quad c_{n-k} = \dfrac{-1}{k}\text{Tr}(\mathbf{AM}_k) \quad(k = 1\cdots n)$$

Then, the inverse of the matrix $\textbf{A}$ is given by :
$$\mathbf{A}^{-1} = -\dfrac{1}{c_0} \mathbf{M_n}$$

In [5]:
# FLV Algorithm - Main Function

def FLV(A):
    n = np.shape(A)[0]
    c = np.zeros(n + 1)
    I = np.eye(n)
    for k in range(n + 1):
        if(k == 0):
            M = np.zeros_like(A, dtype=float)
            c[-1] = 1.
        else:
            M = np.matmul(A, M) + c[n - k + 1] * I
            c[n - k] = (-1 / k) * np.trace(np.matmul(A, M))
    inv = (-1 / c[0]) * M
    return inv

---
# Bringing it all Together

In [6]:
matrix = np.array([[5, 7, 6, 3], [7, 10, 8, 7], [6, 8, 10, 9], [5, 7, 9, 10]], dtype=float)
inv1 = traditional_matrix_inverse(matrix)
inv2 = nontraditional_matrix_inverse(matrix)
inv3 = FLV(matrix)


In [7]:
np.allclose(np.matmul(matrix, inv1) , np.eye(np.shape(inv1)[0]))

True

In [8]:
np.allclose(np.matmul(matrix, inv2) , np.eye(np.shape(inv2)[0]))

True

In [9]:
np.allclose(np.matmul(matrix, inv3) , np.eye(np.shape(inv3)[0]))

True

In [10]:
determinant(matrix)

-19.0

In [11]:
inv1

array([[-3.57894737,  1.94736842,  4.47368421, -4.31578947],
       [ 2.15789474, -0.89473684, -2.94736842,  2.63157895],
       [ 0.89473684, -0.73684211, -0.36842105,  0.57894737],
       [-0.52631579,  0.31578947,  0.15789474, -0.10526316]])

In [12]:
inv2

array([[-3.57894737,  1.94736842,  4.47368421, -4.31578947],
       [ 2.15789474, -0.89473684, -2.94736842,  2.63157895],
       [ 0.89473684, -0.73684211, -0.36842105,  0.57894737],
       [-0.52631579,  0.31578947,  0.15789474, -0.10526316]])

In [13]:
inv3

array([[-3.57894737,  1.94736842,  4.47368421, -4.31578947],
       [ 2.15789474, -0.89473684, -2.94736842,  2.63157895],
       [ 0.89473684, -0.73684211, -0.36842105,  0.57894737],
       [-0.52631579,  0.31578947,  0.15789474, -0.10526316]])